# MLOps Zoomcamp 2025 - Homework 2

This notebook contains the solution for the second homework assignment of the MLOps Zoomcamp course.

In [1]:
%config AsyncShell.timeout = 300  # Increase to 300 seconds
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd
import sklearn
import pyarrow
import mlflow

* run the following command to start the MLflow UI server:


```bash
mlflow ui --backend-store-uri sqlite:///mlflow.db --host 0.0.0.0 --port 5000
```
Works will at github code spaces

In [3]:
# Directory paths 
SCRIPT_PATH = "scripts"
RAW_DATA_PATH = "data"
VECTORIZED_DATA_PATH = "dataset_as_DictVectorizer"

In [2]:
import mlflow
import sqlite3

# Here we set the tracking URI to a local SQLite database named "mlflow.db".
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Next we set the experiment name to "nyc-taxi-experiment".
# First, let's delete the experiment permanently from the database

# Connect to the SQLite database
conn = sqlite3.connect("mlflow.db")
cursor = conn.cursor()

# Delete the experiment from the experiments table
cursor.execute("DELETE FROM experiments WHERE name = ?", ("nyc-taxi-experiment",))

# Commit the changes and close the connection
conn.commit()
conn.close()

# Now we can create a new experiment with the same name
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/26 19:36:05 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/4', creation_time=1748288165031, experiment_id='4', last_update_time=1748288165031, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Q1. What is the version of MlFlow that we currently use?

In [5]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data

In [16]:
# Download the data for Green Taxi Trips from 2023 january to March
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

# Moving the downloaded files to the data directory
!mkdir -p data
!mv green_tripdata_2023-01.parquet data/
!mv green_tripdata_2023-02.parquet data/
!mv green_tripdata_2023-03.parquet data/

--2025-05-25 16:02:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.163, 18.239.38.181, 18.239.38.83, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  --.-KB/s    in 0.01s   

2025-05-25 16:02:05 (137 MB/s) - ‘green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2025-05-25 16:02:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.181, 18.239.38.83, 18.239.38.147, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.181|:443... connected.
HTTP request sent, 

### Preprocessing the data using the `preprocess_data` function

In [4]:
# Preprocess the data and save it as a DictVectorizer object
!python {SCRIPT_PATH}/preprocess_data.py --raw_data_path {RAW_DATA_PATH} --dest_path {VECTORIZED_DATA_PATH}

In [5]:
import os

# Get list of files in the directory
files = os.listdir(f"{VECTORIZED_DATA_PATH}")
file_count = len(files)

print(f"Number of files in dataset_as_DictVectorizer: {file_count}")
print("\nFiles in the directory:")
for file in files:
    print(f"- {file}")

Number of files in dataset_as_DictVectorizer: 4

Files in the directory:
- train.pkl
- val.pkl
- dv.pkl
- test.pkl


## Q3. Train a model with autolog

What is the value of the min_samples_split parameter?


use the train scirpt to do experiment tracking using MLflow

```bash 
python train.py --data_path ./dataset_as_DictVectorizer --model_path models/model.pkl
```

In [8]:
!python {SCRIPT_PATH}/train.py --data_path {VECTORIZED_DATA_PATH}

Validation RMSE: 5.431162180141208


#### The value of parameter `min_samples_split` is set to 2, from stored parameters in the MLflow UI.  
![MLflow UI screenshot showing the parameter configuration with min_samples_split highlighted and set to value 2 in the model parameters section. The interface displays the experiment tracking details for the NYC taxi trip duration prediction model.](./images/mlops_zoomcamp_q3.png)

## Q4. Launch the tracking server locally


what else do you need to pass to properly configure the server?

* `default-artifact-root`  
* `serve-artifacts`  
* `artifacts-only`  
* `artifacts-destination`  

In [ ]:
!mlflow server --backend-store-uri sqlite:///mlruns.db --default-artifact-root ./artifacts
# The answer is default-artifact-root

## Q5. Tune model hyperparameters

What's the best validation RMSE that you got?

In [9]:
!python {SCRIPT_PATH}/hpo.py --data_path {VECTORIZED_DATA_PATH}

2025/05/26 20:10:12 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run exultant-shoat-85 at: http://127.0.0.1:5000/#/experiments/2/runs/77ac45d3787546fe92b93b0c82476188

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run learned-bass-18 at: http://127.0.0.1:5000/#/experiments/2/runs/a08e80a337084a32bbf26e769acca7bf

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run bright-panda-651 at: http://127.0.0.1:5000/#/experiments/2/runs/7fdc30ad8e024e29afb75e826c261327

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run redolent-mule-433 at: http://127.0.0.1:5000/#/experiments/2/runs/d3e3deb29f8a47e5822f156b90754888

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run honorable-sheep-231 at: http://127.0.0.1:5000/#/experiments/2/runs/934c769382de4774863790a

After hyperparameter optimization with different model configurations, the best validation RMSE achieved was 5.335419588556921.

## Q6. Promote the best model to the model registry

In [10]:
!python {SCRIPT_PATH}/register_model.py --data_path {VECTORIZED_DATA_PATH}

2025/05/26 20:18:45 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models-v2' does not exist. Creating a new experiment.
🏃 View run judicious-croc-890 at: http://127.0.0.1:5000/#/experiments/3/runs/187ca8ebf7364cfba5f69ba90c75c3b3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run carefree-donkey-755 at: http://127.0.0.1:5000/#/experiments/3/runs/1429eba1016541d28333a61581a1cc5c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run bemused-stag-609 at: http://127.0.0.1:5000/#/experiments/3/runs/346fe1d69e4045bc8d63899a132cce51
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run judicious-gnat-347 at: http://127.0.0.1:5000/#/experiments/3/runs/477e1e70ec924c999503eaaa3b3b0ecd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run agreeable-rat-411 at: http://127.0.0.1:5000/#/experiments/3/runs/4205a74fe5954151becfbe5aca69c454
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
Successf

The test RMSE of the best model after promotion to the registry is 5.567. This represents our final model performance on unseen data.

![Random Forest Best Models V2](./images/random_forest_best_models_v2.png)